# Task 2: Build Time Series Forecasting Models (TSLA)

## Objective
Develop, train, and evaluate time series forecasting models to predict Tesla's future stock prices.

**Models:**
- ARIMA/SARIMA (via pmdarima)
- LSTM (Keras)

**Metrics:**
- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- MAPE (Mean Absolute Percentage Error)

## 1. Imports and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print('TensorFlow version:', tf.__version__)
print('Setup complete!')

## 2. Load Data (TSLA Adj Close)

In [ ]:
prices = pd.read_csv('../data/processed/adj_close_prices.csv', parse_dates=['Date'], index_col='Date')
prices = prices.sort_index()

tsla = prices['TSLA'].dropna().astype(float)

print('TSLA series shape:', tsla.shape)
print('Date range:', tsla.index.min().date(), 'to', tsla.index.max().date())
print('Sample values:')
tsla.tail()

## 3. Train/Test Split (Chronological)

- **Train:** 2015-01-01 → 2024-12-31
- **Test:** 2025-01-01 → end of data

In [ ]:
split_date = '2024-12-31'

train = tsla.loc[:split_date].copy()
test = tsla.loc[split_date:].iloc[1:].copy()  # Exclude split_date itself from test

print(f'Train: {train.index.min().date()} → {train.index.max().date()} | n = {len(train)}')
print(f'Test:  {test.index.min().date()} → {test.index.max().date()} | n = {len(test)}')

# Sanity checks
assert len(train) > 0 and len(test) > 0, 'Train or test is empty!'
assert train.index.max() < test.index.min(), 'Data leakage: train/test overlap!'

## 4. Metrics Utilities

In [ ]:
def calc_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def calc_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = y_true != 0
    return np.mean(np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero])) * 100

def evaluate(name, y_true, y_pred):
    """Compute metrics. Inputs can be arrays or Series."""
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # Remove any NaN pairs
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true, y_pred = y_true[mask], y_pred[mask]
    
    if len(y_true) == 0:
        return {'Model': name, 'MAE': np.nan, 'RMSE': np.nan, 'MAPE (%)': np.nan, 'N': 0}
    
    return {
        'Model': name,
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': calc_rmse(y_true, y_pred),
        'MAPE (%)': calc_mape(y_true, y_pred),
        'N': len(y_true)
    }

print('Metrics functions defined.')

## 5. Baseline Model (Naive Persistence)

Predict: tomorrow's price = today's price

In [ ]:
# For test period, naive prediction is the previous day's actual value
naive_pred = test.shift(1).dropna()
naive_actual = test.loc[naive_pred.index]

baseline_metrics = evaluate('Naive (t-1)', naive_actual, naive_pred)
print('Baseline metrics:', baseline_metrics)

## 6. ARIMA Model

Use `auto_arima` to find optimal (p,d,q), then use statsmodels ARIMA for forecasting.

In [ ]:
print('Finding optimal ARIMA parameters with auto_arima...')
print('This may take a few minutes...\n')

auto_model = auto_arima(
    train.values,
    start_p=0, max_p=5,
    start_q=0, max_q=5,
    d=None,  # Let it determine d
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    trace=True,
    n_fits=50
)

best_order = auto_model.order
print(f'\nBest ARIMA order: {best_order}')
print(auto_model.summary())

In [ ]:
# Fit ARIMA using statsmodels for more control
print(f'Fitting ARIMA{best_order} on training data...')

arima_model = ARIMA(train.values, order=best_order)
arima_fit = arima_model.fit()

print('Model fitted successfully!')
print(arima_fit.summary())

In [ ]:
# Forecast for test period
n_forecast = len(test)
forecast_result = arima_fit.get_forecast(steps=n_forecast)
arima_pred_values = forecast_result.predicted_mean

# Create Series with proper index
arima_pred = pd.Series(arima_pred_values, index=test.index, name='ARIMA_Pred')

print(f'ARIMA predictions shape: {arima_pred.shape}')
print(f'NaN count in predictions: {arima_pred.isna().sum()}')
print('\nFirst few predictions:')
print(arima_pred.head())
print('\nActual test values:')
print(test.head())

In [ ]:
arima_metrics = evaluate('ARIMA', test.values, arima_pred.values)
print('ARIMA metrics:', arima_metrics)

## 7. LSTM Model

Steps:
1. Scale data (fit on train only)
2. Create sequences with lookback window
3. Train LSTM
4. Predict on test (one-step-ahead using true history)

In [ ]:
LOOKBACK = 60

# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))

print(f'Train scaled shape: {train_scaled.shape}')
print(f'Scaler data min: {scaler.data_min_[0]:.2f}, max: {scaler.data_max_[0]:.2f}')

In [ ]:
def create_sequences(data, lookback):
    """Create sequences for LSTM: X[i] = data[i-lookback:i], y[i] = data[i]"""
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, LOOKBACK)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Validation split (last 10% of training sequences)
val_size = max(1, int(0.1 * len(X_train)))
X_tr, X_val = X_train[:-val_size], X_train[-val_size:]
y_tr, y_val = y_train[:-val_size], y_train[-val_size:]

print(f'X_train shape: {X_train.shape}')
print(f'Training: {X_tr.shape}, Validation: {X_val.shape}')

In [ ]:
# Build LSTM model
keras.backend.clear_session()

lstm_model = keras.Sequential([
    keras.layers.Input(shape=(LOOKBACK, 1)),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(32),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

In [ ]:
# Train with early stopping
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = lstm_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

print(f'\nTraining stopped at epoch {len(history.history["loss"])}')

In [ ]:
# Plot training curves
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('LSTM Training Curve')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# One-step-ahead predictions on test set
# For each test day, use the previous LOOKBACK days (from actual data) to predict

full_data = pd.concat([train, test])
full_scaled = scaler.transform(full_data.values.reshape(-1, 1))

lstm_preds = []
lstm_dates = []

train_len = len(train)
for i in range(train_len, len(full_data)):
    # Get lookback window
    start_idx = i - LOOKBACK
    if start_idx < 0:
        continue
    
    window = full_scaled[start_idx:i, 0]
    X_input = window.reshape(1, LOOKBACK, 1)
    
    pred_scaled = lstm_model.predict(X_input, verbose=0)[0, 0]
    lstm_preds.append(pred_scaled)
    lstm_dates.append(full_data.index[i])

# Inverse transform predictions
lstm_pred_values = scaler.inverse_transform(np.array(lstm_preds).reshape(-1, 1)).flatten()
lstm_pred = pd.Series(lstm_pred_values, index=pd.DatetimeIndex(lstm_dates), name='LSTM_Pred')

print(f'LSTM predictions: {len(lstm_pred)}')
print(lstm_pred.head())

In [ ]:
# Align with test for evaluation
lstm_test_actual = test.loc[lstm_pred.index]

lstm_metrics = evaluate('LSTM', lstm_test_actual.values, lstm_pred.values)
print('LSTM metrics:', lstm_metrics)

## 8. Model Comparison

In [ ]:
results = pd.DataFrame([baseline_metrics, arima_metrics, lstm_metrics])
results = results.sort_values(by='RMSE').reset_index(drop=True)

print('=' * 60)
print('MODEL COMPARISON (sorted by RMSE)')
print('=' * 60)
results

In [ ]:
# Visualization: Test period predictions
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Full view
ax1 = axes[0]
ax1.plot(train.index, train.values, label='Train', linewidth=1, alpha=0.7)
ax1.plot(test.index, test.values, label='Test (Actual)', linewidth=1.5, color='black')
ax1.plot(arima_pred.index, arima_pred.values, label='ARIMA', linewidth=1.5, linestyle='--')
ax1.plot(lstm_pred.index, lstm_pred.values, label='LSTM', linewidth=1.5, linestyle='--')
ax1.set_title('TSLA Price: Full History + Forecasts')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Zoomed to test period
ax2 = axes[1]
ax2.plot(test.index, test.values, label='Actual', linewidth=2, color='black', marker='o', markersize=3)
ax2.plot(arima_pred.index, arima_pred.values, label='ARIMA', linewidth=1.5, linestyle='--', marker='s', markersize=3)
ax2.plot(lstm_pred.index, lstm_pred.values, label='LSTM', linewidth=1.5, linestyle='--', marker='^', markersize=3)
ax2.set_title('TSLA Price: Test Period (Zoomed)')
ax2.set_xlabel('Date')
ax2.set_ylabel('Price ($)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Model Selection & Discussion

Based on the metrics above, select the best model for Task 3 forecasting.

In [ ]:
best_model_name = results.iloc[0]['Model']
print('=' * 60)
print(f'BEST MODEL (by RMSE): {best_model_name}')
print('=' * 60)
print(results.iloc[0])

print('\n--- Discussion ---')
print('''
Model Selection Rationale:

1. ARIMA:
   - Pros: Interpretable, provides confidence intervals natively
   - Cons: Assumes linear relationships, may struggle with complex patterns

2. LSTM:
   - Pros: Can capture non-linear patterns and long-term dependencies
   - Cons: Requires more data, harder to interpret, no native CI

3. Naive Baseline:
   - Useful benchmark; if models don't beat this, they add no value

For Task 3 (future forecasting with confidence intervals), ARIMA is often 
preferred due to built-in uncertainty quantification.
''')

## 10. Save Artifacts

In [ ]:
import os
import joblib

os.makedirs('../data/processed/models', exist_ok=True)

# Save LSTM model and scaler
lstm_model.save('../data/processed/models/tsla_lstm.keras')
joblib.dump(scaler, '../data/processed/models/tsla_scaler.joblib')

# Save ARIMA info
arima_info = {
    'order': best_order,
    'aic': arima_fit.aic,
    'bic': arima_fit.bic
}
joblib.dump(arima_info, '../data/processed/models/tsla_arima_info.joblib')

# Save results table
results.to_csv('../data/processed/models/model_comparison.csv', index=False)

print('Artifacts saved to data/processed/models/')
print('  - tsla_lstm.keras')
print('  - tsla_scaler.joblib')
print('  - tsla_arima_info.joblib')
print('  - model_comparison.csv')

In [ ]:
print('\n' + '=' * 60)
print('TASK 2 COMPLETE')
print('=' * 60)
print('\nDeliverables:')
print('  ✓ Trained ARIMA model with documented parameters')
print('  ✓ Trained LSTM model with documented architecture')
print('  ✓ Model comparison table with MAE, RMSE, MAPE')
print('  ✓ Discussion of model selection rationale')
print('  ✓ Artifacts saved for Task 3')